<a href="https://colab.research.google.com/github/bartaugust/Detekcja-anomalii/blob/main/Lab1_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import random
import numpy as np

import torch
import torchvision

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

In [ ]:
!gdown 1YSs2XjzvAM_0ZbttiznTDo7GUdCX1erK

Downloading...
From: https://drive.google.com/uc?id=1YSs2XjzvAM_0ZbttiznTDo7GUdCX1erK
To: /content/ResNet152_CIFAR10.ckpt
100% 466M/466M [00:02<00:00, 182MB/s]


In [ ]:
model = torchvision.models.resnet152(num_classes=10)
        
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()

preprocess = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

state_dict = torch.load("./ResNet152_CIFAR10.ckpt", map_location='cuda:0')["state_dict"] 
for key in list(state_dict.keys()):
    state_dict[key.replace('model.', '')] = state_dict.pop(key)
    
model.load_state_dict(state_dict)    
model.cuda()
model.eval()
print("Model loaded")

Model loaded


In [ ]:
test_set = torchvision.datasets.CIFAR10(root="./", train=False, download=True, transform=preprocess)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

total, correct = 0, 0
with torch.no_grad():
  for i_batch, (data, targets) in enumerate(test_loader):                
      data, targets = data.cuda(), targets.cuda()
      logits = model(data)
      _, predicted = torch.max(logits, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

print("Acc", correct/total)

Files already downloaded and verified
Acc 0.953


In [ ]:
def get_features(model, data):
    print(data.shape)
    for name, module in model._modules.items():
        data = module(data)
        print(name, data.shape)
        #...
        #return ...

data, targets = next(iter(test_loader))
print(data.shape)
print(targets.shape, targets[0])
print("---")
get_features(model, data.cuda())

torch.Size([32, 3, 32, 32])
torch.Size([32]) tensor(3)
---
torch.Size([32, 3, 32, 32])
conv1 torch.Size([32, 64, 32, 32])
bn1 torch.Size([32, 64, 32, 32])
relu torch.Size([32, 64, 32, 32])
maxpool torch.Size([32, 64, 32, 32])
layer1 torch.Size([32, 256, 32, 32])
layer2 torch.Size([32, 512, 16, 16])
layer3 torch.Size([32, 1024, 8, 8])
layer4 torch.Size([32, 2048, 4, 4])
avgpool torch.Size([32, 2048, 1, 1])


RuntimeError: ignored